In [7]:
import psycopg2
import pandas as pd

In [8]:
host = 'materialserver-3163.j77.aws-ap-southeast-1.cockroachlabs.cloud'
port = '26257'
dbname = 'material'
user = 'vendor'
password = 'k_wEbjoSUNNXFzdjxdzxTg'

conn = psycopg2.connect(
    host=host,
    port=port,
    database=dbname,
    user=user,
    password=password
)


In [9]:
material = pd.read_sql('SELECT * FROM materialhierarchy', conn)
material

C:\Users\Admin\AppData\Local\Temp\ipykernel_10876\2526059148.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  material = pd.read_sql('SELECT * FROM materialhierarchy', conn)


,ProductAssemblyID,ComponentID,TotalQuantity,BOMLevel,RecursionLevel
0,3,2,10.0,3,0
1,3,504,2.0,3,0
2,3,461,1.0,3,2
3,3,505,2.0,3,2
4,316,486,1.0,4,0
...,...,...,...,...,...
2378,999,945,1.0,1,0
2379,999,948,1.0,1,0
2380,999,949,1.0,1,0
2381,999,952,1.0,1,0


In [ ]:
def calculate_component_quantities(df, product_id, initial_quantity):
    result = []

    def recurse(current_id, current_quantity, current_level):
        components = df[df["ProductAssemblyID"] == current_id]
        for _, row in components.iterrows():
            component_id = row["ComponentID"]
            total_quantity = current_quantity * row["TotalQuantity"]
            result.append({
                "ComponentID": component_id,
                "TotalQuantity": total_quantity,
                "Level": current_level + 1
            })

            recurse(component_id, total_quantity, current_level + 1)

    recurse(product_id, initial_quantity, 0)

    result_df = pd.DataFrame(result)
    aggregated = result_df.groupby(["ComponentID", "Level"]).sum().reset_index()

    return aggregated

In [ ]:
def calculate_leaf_component_quantities(df, product_id, initial_quantity):
    result = []

    def recurse(current_id, current_quantity, current_level):
        components = df[df["ProductAssemblyID"] == current_id]
        
        if components.empty:
            result.append({
                "ComponentID": current_id,
                "TotalQuantity": current_quantity,
                "Level": current_level
            })
            return
        
        for _, row in components.iterrows():
            component_id = row["ComponentID"]
            total_quantity = current_quantity * row["TotalQuantity"]
            recurse(component_id, total_quantity, current_level + 1)

    recurse(product_id, initial_quantity, 0)

    result_df = pd.DataFrame(result)

    return result_df

In [ ]:
materialhierarchy = pd.DataFrame(material)

input_product_id = 994
input_quantity = 10

result_df = calculate_leaf_component_quantities(materialhierarchy, input_product_id, input_quantity)

print(result_df)

   ComponentID  TotalQuantity  Level
0          2.0         1000.0      2
1        504.0          200.0      2
2        461.0          100.0      2
3        505.0          200.0      2
4        525.0           10.0      1


In [ ]:
result2_df = calculate_component_quantities(materialhierarchy, input_product_id, input_quantity)

print(result2_df)